# Retrieve World Bank Trade Data

Data comes from [https://wits.worldbank.org/witsapiintro.aspx]

Prereqs:
* Need to run "pip install pandaSDMX" to install parser to read SDMX data from a request

In [1]:
from pandasdmx import Request
import json
from pprint import pprint as pp
import pandas as pd
import numpy as np

In [2]:
# data is in thousands of USD, so dividing by 1 million makes the values in billions
million = 1000000

wits = Request('WBG_WITS')
flow_response = wits.dataflow()
flow_response.write().dataflow.head()

,name
dataflow,
DF_WITS_Tariff_TRAINS,WITS - UNCTAD TRAINS Tariff Data
DF_WITS_TradeStats_Development,WITS TradeStats Devlopment
DF_WITS_TradeStats_Tariff,WITS TradeStats Tariff
DF_WITS_TradeStats_Trade,WITS TradeStats Trade


In [3]:
trade_stats = wits.dataflow('DF_WITS_TradeStats_Trade')

In [4]:
trade_data_structure = trade_stats.dataflow.DF_WITS_TradeStats_Trade.structure()
trade_data_structure

DataStructureDefinition | TRADESTATS | Country Trade Statistics

In [5]:
trade_data_structure.dimensions.aslist()

[Dimension | FREQ,
 Dimension | REPORTER,
 Dimension | PARTNER,
 TimeDimension | TIME_PERIOD,
 Dimension | PRODUCTCODE,
 Dimension | INDICATOR]

Only interested in trade with the US

In [6]:
reporters = trade_stats.write().codelist.loc['REPORTER']
select_reporter = reporters.loc[['USA']]
select_reporter

,dim_or_attr,name
USA,D,United States


Looking for specific trade partners: Mexico, Canada, EU, China, Russia, World. We limit the definition of the EU to just the 11 countries that adopted the Euro as their currency in 1999.  I tried to use the partner code 'EUN' but there is no data in the with that partner code.

In [7]:
partners = trade_stats.write().codelist.loc['PARTNER']

#for index, row in partners.iterrows():
#    print(f"'{index}':\t\t'{row['name']}'")

# Countries we are interested
partner_list = ['MEX','CAN','RUS','JPN','KOR','CHN']
          
# Countries which adopted the Euro as their currency in 1999
eu_list = ['AUT','BEL','FIN','FRA','DEU','IRL','ITA','LUX','NLD','PRT','ESP']
          
# Separate list for the lonely total partner representing the world 'WLD'
world_list = ['WLD']
          
select_partners = partners.loc[partner_list + eu_list + world_list]
select_partners

,dim_or_attr,name
MEX,D,Mexico
CAN,D,Canada
RUS,D,Russian Federation
JPN,D,Japan
KOR,D,"Korea, Rep."
CHN,D,China
AUT,D,Austria
BEL,D,Belgium
FIN,D,Finland
FRA,D,France


Only product we are interested in is "all products" i.e. 'Total'

In [8]:
products = trade_stats.write().codelist.loc['PRODUCTCODE']

#for index, row in products.iterrows():
#    print(f"'{index}':\t\t'{row['name']}'")

select_product = products.loc[['Total']]
select_product

,dim_or_attr,name
Total,D,All Products


Only looking for the import and export values. All values for these two indicators are in thousands of USD.

In [9]:
indicators = trade_stats.write().codelist.loc['INDICATOR']

#for index, row in indicators.iterrows():
#    print(f"'{index}':\t\t'{row['name']}'")

select_indicators = indicators.loc[['MPRT-TRD-VL', 'XPRT-TRD-VL']]
select_indicators

,dim_or_attr,name
MPRT-TRD-VL,D,Import Trade Value (US$ Thousand)
XPRT-TRD-VL,D,Export Trade Value (US$ Thousand)


Only time frequency is 'annual'.

In [10]:
trade_stats.write().codelist.loc['FREQ'].head()

,dim_or_attr,name
FREQ,D,Freq
A,D,Annual


In [11]:
freq = 'A'
partners_list = select_partners.index.tolist()
reporter = select_reporter.index.tolist()[0]
product = select_product.index.tolist()[0]
indicator_list = select_indicators.index.tolist()

response = wits.data(resource_id = 'DF_WITS_TradeStats_Trade',
                 key={'FREQ': freq,
                      'REPORTER': reporter,
                      'PARTNER':  partners_list,
                      'PRODUCTCODE': product,
                      'INDICATOR': indicator_list},
                 params = {'startPeriod': '2000',
                           'endPeriod': '2017'},
                 dsd = trade_data_structure)

In [12]:
trade_df = response.write((s for s in response.data.series))
trade_df.rename(columns={'MPRT-TRD-VL': 'Imports', 'XPRT-TRD-VL': 'Exports'}, inplace=True)

# Scale all values into billions
trade_df = trade_df / million

In [13]:
import_columns_to_sum = [ ('A', 'USA', cty, 'Total', 'Imports') for cty in eu_list ]
import_column_eu = ('A', 'USA', 'EU', 'Total', 'Imports')
trade_df[import_column_eu] = trade_df[import_columns_to_sum].sum(axis=1)
trade_df.drop(import_columns_to_sum, axis=1, inplace=True)

export_columns_to_sum = [ ('A', 'USA', cty, 'Total', 'Exports') for cty in eu_list ]
export_column_eu = ('A', 'USA', 'EU', 'Total', 'Exports')
trade_df[export_column_eu] = trade_df[export_columns_to_sum].sum(axis=1)
trade_df.drop(export_columns_to_sum, axis=1, inplace=True)

known_cty_list = partner_list + [ 'EU' ]

known_import_columns_to_sum = [ ('A', 'USA', cty, 'Total', 'Imports') for cty in known_cty_list ]
import_column_other = ('A', 'USA', 'Other', 'Total', 'Imports')
import_column_world = ('A', 'USA', 'WLD', 'Total', 'Imports')
trade_df[import_column_other] = trade_df[import_column_world] - trade_df[known_import_columns_to_sum].sum(axis=1)

known_export_columns_to_sum = [ ('A', 'USA', cty, 'Total', 'Exports') for cty in known_cty_list ]
export_column_other = ('A', 'USA', 'Other', 'Total', 'Exports')
export_column_world = ('A', 'USA', 'WLD', 'Total', 'Exports')
trade_df[export_column_other] = trade_df[export_column_world] - trade_df[known_export_columns_to_sum].sum(axis=1)

trade_df.head()

FREQ                  A                                                 \
REPORTER            USA                                                  
PARTNER             CAN                     CHN                    JPN   
PRODUCTCODE       Total                   Total                  Total   
INDICATOR       Imports     Exports     Imports    Exports     Imports   
TIME_PERIOD                                                              
2000         230.816138  178.919985  100.012903  16.184679  146.479387   
2001         216.234127  163.421117  102.267289  19.181936  126.473307   
2002         212.398395  160.918225  133.510393  22.127509  124.566141   
2003         225.035349  169.920170  163.313863  28.367721  121.238585   
2004         260.161286  189.878784  210.500129  34.427687  133.551797   

FREQ                                                                  \
REPORTER                                                               
PARTNER                       KOR                    MEX               
PRODUCTCODE                 Total                  Total               
INDICATOR      Exports    Imports    Exports     Imports     Exports   
TIME_PERIOD                                                            
2000         64.921645  40.307624  27.829956  135.923121  111.338635   
2001         57.449648  35.181331  22.180578  131.334695  101.295130   
2002         51.447934  36.906553  22.575714  136.025056   97.470195   
2003         52.003765  38.616672  24.072540  139.686658   97.411740   
2004         53.568570  47.819862  26.186728  157.881924  110.731247   

FREQ                                                                   \
REPORTER                                                                
PARTNER            RUS                    WLD                      EU   
PRODUCTCODE      Total                  Total                   Total   
INDICATOR      Imports   Exports      Imports     Exports     Imports   
TIME_PERIOD                                                             
2000          7.658130  2.092380  1217.932974  781.830673  163.543932   
2001          6.264073  2.716088  1140.900159  729.080421  165.886564   
2002          7.207030  2.396944  1200.095834  693.068307  176.681136   
2003          9.106465  2.447218  1302.833508  724.736583  191.704028   
2004         12.618720  2.960937  1525.304217  814.844394  214.841432   

FREQ                                             
REPORTER                                         
PARTNER                       Other              
PRODUCTCODE                   Total              
INDICATOR       Exports     Imports     Exports  
TIME_PERIOD                                      
2000         116.362492  393.191740  264.180900  
2001         111.836514  357.258773  250.999409  
2002         105.010952  372.801131  231.120834  
2003         110.897974  414.131888  239.615454  
2004         124.131796  487.929069  272.958646

In [14]:
# This line commented below should have worked but it doesn't.  You cannot use
# rename to rename columns with tuple names.  I had to use something a little
# more dangerous by explicitly set the columns attribute of the DataFrame

#new_column_names = { x: f"{x[2]}_{x[4]}" for x in list(trade_df) }
#trade_df.rename(columns=new_column_names, inplace=True)

trade_df.columns = [f"{x[2]}_{x[4]}" for x in trade_df.columns]
trade_df.head()

,CAN_Imports,CAN_Exports,CHN_Imports,CHN_Exports,JPN_Imports,JPN_Exports,KOR_Imports,KOR_Exports,MEX_Imports,MEX_Exports,RUS_Imports,RUS_Exports,WLD_Imports,WLD_Exports,EU_Imports,EU_Exports,Other_Imports,Other_Exports
TIME_PERIOD,,,,,,,,,,,,,,,,,,
2000,230.816138,178.919985,100.012903,16.184679,146.479387,64.921645,40.307624,27.829956,135.923121,111.338635,7.658130,2.092380,1217.932974,781.830673,163.543932,116.362492,393.191740,264.180900
2001,216.234127,163.421117,102.267289,19.181936,126.473307,57.449648,35.181331,22.180578,131.334695,101.295130,6.264073,2.716088,1140.900159,729.080421,165.886564,111.836514,357.258773,250.999409
2002,212.398395,160.918225,133.510393,22.127509,124.566141,51.447934,36.906553,22.575714,136.025056,97.470195,7.207030,2.396944,1200.095834,693.068307,176.681136,105.010952,372.801131,231.120834
2003,225.035349,169.920170,163.313863,28.367721,121.238585,52.003765,38.616672,24.072540,139.686658,97.411740,9.106465,2.447218,1302.833508,724.736583,191.704028,110.897974,414.131888,239.615454
2004,260.161286,189.878784,210.500129,34.427687,133.551797,53.568570,47.819862,26.186728,157.881924,110.731247,12.618720,2.960937,1525.304217,814.844394,214.841432,124.131796,487.929069,272.958646


In [15]:
new_partner_list = partner_list + world_list + ['EU','Other']

partner_import_cols = [f"{s}_Imports" for s in new_partner_list]
partner_export_cols = [f"{s}_Exports" for s in new_partner_list]
for p in new_partner_list:
    trade_df[f"{p}_Diff"] = trade_df[f"{p}_Imports"] - trade_df[f"{p}_Exports"]
trade_df.head()

,CAN_Imports,CAN_Exports,CHN_Imports,CHN_Exports,JPN_Imports,JPN_Exports,KOR_Imports,KOR_Exports,MEX_Imports,MEX_Exports,...,Other_Exports,MEX_Diff,CAN_Diff,RUS_Diff,JPN_Diff,KOR_Diff,CHN_Diff,WLD_Diff,EU_Diff,Other_Diff
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
2000,230.816138,178.919985,100.012903,16.184679,146.479387,64.921645,40.307624,27.829956,135.923121,111.338635,...,264.180900,24.584486,51.896152,5.565750,81.557742,12.477668,83.828223,436.102302,47.181440,129.010840
2001,216.234127,163.421117,102.267289,19.181936,126.473307,57.449648,35.181331,22.180578,131.334695,101.295130,...,250.999409,30.039565,52.813010,3.547984,69.023659,13.000752,83.085353,411.819738,54.050050,106.259364
2002,212.398395,160.918225,133.510393,22.127509,124.566141,51.447934,36.906553,22.575714,136.025056,97.470195,...,231.120834,38.554861,51.480170,4.810086,73.118206,14.330839,111.382884,507.027527,71.670184,141.680297
2003,225.035349,169.920170,163.313863,28.367721,121.238585,52.003765,38.616672,24.072540,139.686658,97.411740,...,239.615454,42.274918,55.115179,6.659247,69.234820,14.544132,134.946142,578.096925,80.806054,174.516434
2004,260.161286,189.878784,210.500129,34.427687,133.551797,53.568570,47.819862,26.186728,157.881924,110.731247,...,272.958646,47.150676,70.282502,9.657784,79.983226,21.633134,176.072442,710.459823,90.709636,214.970423


In [16]:
for p in new_partner_list:
    trade_df[f"{p}_Scale"] = trade_df[f"{p}_Diff"] / trade_df[f"{p}_Exports"]
trade_df.head()

,CAN_Imports,CAN_Exports,CHN_Imports,CHN_Exports,JPN_Imports,JPN_Exports,KOR_Imports,KOR_Exports,MEX_Imports,MEX_Exports,...,Other_Diff,MEX_Scale,CAN_Scale,RUS_Scale,JPN_Scale,KOR_Scale,CHN_Scale,WLD_Scale,EU_Scale,Other_Scale
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
2000,230.816138,178.919985,100.012903,16.184679,146.479387,64.921645,40.307624,27.829956,135.923121,111.338635,...,129.010840,0.220808,0.290052,2.660009,1.256249,0.448354,5.179480,0.557796,0.405469,0.488343
2001,216.234127,163.421117,102.267289,19.181936,126.473307,57.449648,35.181331,22.180578,131.334695,101.295130,...,106.259364,0.296555,0.323171,1.306285,1.201464,0.586132,4.331437,0.564848,0.483295,0.423345
2002,212.398395,160.918225,133.510393,22.127509,124.566141,51.447934,36.906553,22.575714,136.025056,97.470195,...,141.680297,0.395555,0.319915,2.006758,1.421208,0.634790,5.033684,0.731569,0.682502,0.613014
2003,225.035349,169.920170,163.313863,28.367721,121.238585,52.003765,38.616672,24.072540,139.686658,97.411740,...,174.516434,0.433982,0.324359,2.721150,1.331342,0.604179,4.757032,0.797665,0.728652,0.728319
2004,260.161286,189.878784,210.500129,34.427687,133.551797,53.568570,47.819862,26.186728,157.881924,110.731247,...,214.970423,0.425812,0.370144,3.261733,1.493100,0.826111,5.114269,0.871896,0.730753,0.787557


So we could use trade_df as our data file or convert the file with partner as a column.  Converting trade_df to trade2_df
to make it easier for tools like Tableau (or Group by's in Pandas)

In [17]:
trade2_df = pd.DataFrame(columns=['Year', 'Partner', 'Imports', 'Exports'])

for idx, value in trade_df.iterrows():
    year_list = []
    for partner in new_partner_list:
        partner_imports = f"{partner}_Imports"
        partner_exports = f"{partner}_Exports"
        year_list.append({
            'Year': idx,
            'Partner': partner,
            'Imports': value[partner_imports],
            'Exports': value[partner_exports]
        })
    trade2_df = trade2_df.append(year_list)
trade2_df.head()

,Year,Partner,Imports,Exports
0,2000,MEX,135.923121,111.338635
1,2000,CAN,230.816138,178.919985
2,2000,RUS,7.658130,2.092380
3,2000,JPN,146.479387,64.921645
4,2000,KOR,40.307624,27.829956


In [18]:
trade2_df['Difference'] = trade2_df['Imports'] - trade2_df['Exports']
trade2_df['Scale'] = trade2_df['Difference'] * 100 / trade2_df['Exports']
trade2_df.head()

,Year,Partner,Imports,Exports,Difference,Scale
0,2000,MEX,135.923121,111.338635,24.584486,22.080822
1,2000,CAN,230.816138,178.919985,51.896152,29.005230
2,2000,RUS,7.658130,2.092380,5.565750,266.000901
3,2000,JPN,146.479387,64.921645,81.557742,125.624884
4,2000,KOR,40.307624,27.829956,12.477668,44.835387


Now let's save the data as JSON so that we do not have to repeatedly make API calls to retrieve the data.

In [19]:
trade_csv = trade2_df.to_csv(index=False)

# now write output to a file
with open('trade_data_file.csv', 'w') as outfile:
    outfile.write(trade_csv)